# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686508


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.40s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:46,  1.72s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:33,  1.30s/it]

Rendering models:  17%|█▋        | 5/30 [00:05<00:25,  1.01s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  3.82it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.98it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  4.70it/s]

Rendering models: 100%|██████████| 30/30 [00:13<00:00,  4.56it/s]

not-logged-in-6cf01ec1c1cb535db647     0.000122
CThomas                                0.000052
Tlalka                                 0.000099
tanya_p                                0.000091
Altrmike                               0.000681
Staceyann116                           0.000378
shocko61                               0.000063
not-logged-in-440db6bf38eb20ce9bea     0.000083
tingard                                0.000076
not-logged-in-7483707cb8546de9ad86     0.000214
not-logged-in-5f30b2bb9b7f9d49da41     0.125745
not-logged-in-f82a6d15a1dafa9bbfae     0.000077
Giantjack14                           39.630565
jmartinez468                           0.020625
AlexYoung35                            0.000202
acapirala                              0.000088
vctjones                               0.048069
aidenr                                 0.000127
Lavadude                               0.051383
layne.woodward2701                     0.000083
Mr_DMan                                0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())